In [60]:
# # Importing the required libraries
import cv2
import glob
import numpy as np
from scipy.stats import multivariate_normal as gaussian
import os
from math import log10, sqrt

In [2]:
!wget https://github.com/JINAY08/Real-TimeTracking/raw/main/WavingTrees.zip  # # Imports the dataset in the form of a zip file.

--2022-11-08 05:58:31--  https://github.com/JINAY08/Real-TimeTracking/raw/main/WavingTrees.zip
Resolving github.com (github.com)... 20.201.28.151
Connecting to github.com (github.com)|20.201.28.151|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JINAY08/Real-TimeTracking/main/WavingTrees.zip [following]
--2022-11-08 05:58:32--  https://raw.githubusercontent.com/JINAY08/Real-TimeTracking/main/WavingTrees.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14579274 (14M) [application/zip]
Saving to: ‘WavingTrees.zip’

WavingTrees.zip     100%[===================>]  13.90M  --.-KB/s    in 0.1s    

2022-11-08 05:58:32 (118 MB/s) - ‘WavingTrees.zip’ saved [14579274/14579274]



In [3]:
!unzip -q WavingTrees -d "/content/WavingTrees/"  # # Unzipping the zip file

In [4]:
os.makedirs('content/WavingTrees/', exist_ok =  True) # # Makes a folder WavingTrees

for filepath in sorted(glob.glob(f'WavingTrees/b*')):   # # # Reads all those files that start with a 'b'
    # print(filepath)
    c = cv2.VideoCapture(filepath) 
    j = 0
    filepathlist = filepath.split('.')
    p, q = c.read()
    filepathlist.append(str(j))
    
    # # # The following block converts the images from .pgm format to .jpg format.
    while p != False:
      filepathlist.pop(-1)
      filepathlist.append(str(j))
      j+=1
      filepath = '.'.join(filepathlist) + '.jpg'
      cv2.imwrite('/content' + filepath, q)
      p, q = c.read()

In [5]:
# # # Defining the Gaussian Function
def gaussiandef(x, mean, s):
  mean = np.atleast_2d(mean)
  s = np.atleast_2d(s)
  N = x.size

  temp1 = np.linalg.det(s) ** (-0.5)
  temp2 = np.exp(-0.5 * ((x - mean).T) @ np.linalg.inv(s) @ (x - mean))
  return (2 * np.pi) ** (-N/2) * temp1 * temp2

In [6]:
# # # Makes a list of paths of all frames
images = glob.glob("/content/WavingTrees/b*.bmp")
# print(images[0].split(".")[0].split("b")[1])
images = sorted(images)
print(f'Total number of images: {len(images)}')
# print(images[200])

Total number of images: 287


In [7]:
# # # Initial set of images
train_set_len = 200
train_set = images[:train_set_len]
test_set = images[train_set_len:]
test_set_len = len(test_set)
print(f'Training set length: {train_set_len}')
print(f'Test set length: {test_set_len}')

Training set length: 200
Test set length: 87


In [8]:
example_img = cv2.imread(train_set[0])
height, width, channels = example_img.shape
print(f'Dimensions of an Image: {height} x {width} x {channels}') # # # Dimensions of each image

Dimensions of an Image: 120 x 160 x 3


In [9]:
alpha = 1.0/train_set_len # # Defining the parameter 'alpha'
K = 5 # # Parameter K defined in the paper (number of Gaussians)
thres1 = 2.5 # # Threshold to classify a pixel as background or foreground
forethres = 0.5 # # Threshold for the Euclidean Distance (to check if it has significant weight in representation of the pixel)

In [10]:
B = np.ones((height, width),dtype=np.int64) # # Matrix for background pixels
F = np.ones((height, width),dtype=np.int64)
weights = np.ndarray(shape=(height,width, K), dtype=np.int64) # # Weight matrix to represent the weights of each of the Gaussian
for h in range(height):
  for w in range(width):
    weights[h,w,:] = [1,0,0,0,0]  # # # Initial weights

print(f'Shape of weight array: {weights.shape}')

Shape of weight array: (120, 160, 5)


In [11]:
# # # Defining the expectation (or the mean) matrix
exp = np.ndarray(shape=(height,width,K,channels), dtype = np.float64)
for h in range(height):
  for w in range(width):
    for k in range(K):
      exp[h,w,k,:] = np.zeros(3)
      exp[h][w][k] = np.array(example_img[h][w]).reshape(1,3)

print(f'Shape of expectation(mean) array: {exp.shape}')

Shape of expectation(mean) array: (120, 160, 5, 3)


In [12]:
# # # Defining the variance matrix
sigma = np.ndarray(shape=(height,width,K,channels, channels), dtype = np.float64)
for h in range(height):
  for w in range(width):
    for k in range(K):
      sigma[h,w,k,:,:] = 255*np.eye(channels)

print(f'Shape of standard deviation array: {sigma.shape}')

Shape of standard deviation array: (120, 160, 5, 3, 3)


In [13]:
# # # Using the first threshold, checks if the pixel could be classified as background or foreground.
def match_check(x, exp, sigma):
    m = np.mat(np.reshape(x, (3, 1)))
    n = np.mat(exp).T
    sigma = np.mat(sigma)
    d = np.sqrt((m-n).T*sigma.I*(m-n))
    if d < thres1:
        return True
    else:
        return False

In [14]:
for training_image in train_set:
  image = cv2.imread(training_image)
  for h in range(height):
    for w in range(w):
      flag = -1
      for k in range(K): # # # To identify which of the Gaussians contribute most significantly to the pixel.
        if (match_check(image[h][w], exp[h][w][k], sigma[h][w][k])) is True:
          flag = k
          break
      m = np.array(image[h][w]).reshape(1,3)

      if flag != 1: # # # If the flag is not equal to 1, we update the model parameters, as given in the paper.
        updated_exp = exp[h][w][flag]
        updated_sigma = sigma[h][w][flag]
        m = image[h][w].astype(np.float)
        p = gaussian.pdf(image[h][w], updated_exp, updated_sigma)
        # p = gaussiandef(image[h][w], updated_exp, updated_sigma)
        weights[h][w] = (1-alpha)*weights[h][w]
        weights[h][w][flag] += alpha
        exp[h][w][flag] += (m - updated_exp)*p
        sigma[h][w][flag] += p*(np.matmul(m-updated_exp, (m-updated_exp).T) - updated_sigma)

      else:
        w = [weights[h][w][k] for k in range(K)] # # # Else, we find the weight matrix!
        minindex = w.index(min(w))
        exp[h][w][minindex] = m
        sigma[h][w][minindex] = 255*(np.eye(3))

  for h in range(height):
        for w in range(width):
            r = weights[h][w]*1.0/[np.linalg.norm(np.sqrt(sigma[h][w][k])) for k in range(K)] # # # Used to identify and sort the Gaussians according to their "significance" in representing the pixel.
            r_index = [k for k in range(K)]
            r_index.sort(key=lambda x: -r[x])
            weights[h][w] = weights[h][w][r_index]
            exp[h][w] =  exp[h][w][r_index]
            sigma[h][w] = sigma[h][w][r_index]
            cum_sum = 0
            for ind, order in enumerate(r_index):
                cum_sum += weights[h][w][ind]
                if cum_sum > forethres:
                    B[h][w] = ind + 1 # # # If the cumulative sum is greater than threshold, classify it as a part of background.
                    break
                else:
                    F[h][w] = ind + 1
                    break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app


In [15]:
# # # This generates the output matrix and sets all bacckground pixels to be black!
def infer(img):
      result = np.array(img)
      for i in range(height):
          for j in range(width):
              for k in range(B[i][j]):
                  if match_check(img[i][j], exp[i][j][k], sigma[i][j][k]):
                      result[i][j] = [0, 0, 0]    
                      break
                  else:
                      result[i][j] = [255, 255, 255]    
                      break
      return result

In [16]:
# # # Forming the output file.
file_list = glob.glob(r'WavingTrees/b*.bmp')
for index, file in enumerate(file_list):
        # print('infering:{}'.format(file))
        img = cv2.imread(file)
        img2 = infer(img)
        cv2.imwrite('output/'+'%05d'%index+'.bmp', img2)
        index += 1

In [17]:
# # # Makes the original video from frames in the dataset.
image_folder = 'WavingTrees/'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".bmp")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
# height, width, layers = frame.shape
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [18]:
# # # Makes the output video from user-defined functions.
out_image_folder = 'output/'
video_name = 'video_out.avi'

images_out = [img for img in os.listdir(out_image_folder) if img.endswith(".bmp")]
frame = cv2.imread(os.path.join(out_image_folder, images_out[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images_out:
    video.write(cv2.imread(os.path.join(out_image_folder, image)))

cv2.destroyAllWindows()
video.release()

Comparison With Inbuilt Function

In [19]:
fgbg = cv2.createBackgroundSubtractorMOG2()

In [20]:
# # # Makes a video from output video generated using the in-built function.
video_name = 'video_from_inbuilt.avi'
image_folder = 'WavingTrees/'
images = [img for img in os.listdir(image_folder) if img.endswith(".bmp")]
for image in range(len(images)):
  frame = cv2.imread(os.path.join(image_folder, images[image]))
  height, width, channels = frame.shape
  fgmask = fgbg.apply(frame)
  cv2.imwrite('output_inbuilt/'+'%05d'%image+'.bmp', fgmask)

out_image_folder = 'output_inbuilt/'
images_out = [img for img in os.listdir(out_image_folder) if img.endswith(".bmp")]
frame = cv2.imread(os.path.join(out_image_folder, images_out[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images_out:
    video.write(cv2.imread(os.path.join(out_image_folder, image)))

cv2.destroyAllWindows()
video.release() 




PSNR

In [21]:
# # # Taken from Geeks for Geeks
def PSNR(original, output):
    mse = np.mean((original - output) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [43]:
out_images = glob.glob("/content/output/*.bmp")
out_images = sorted(out_images)
# print(out_images)
inp_images = glob.glob("/content/WavingTrees/b*.bmp")
inp_images = sorted(inp_images)
# print(inp_images)
PSNR_matrix_defined = [] # # # A matrix that stores the PSNR values of each of the frames.
def max_psnr(original, output):
    original = cv2.imread(original)
    output = cv2.imread(output)
    value = PSNR(original, output)
    return value
for i in range(len(inp_images)):
  PSNR_matrix_defined.append(max_psnr(inp_images[i], out_images[i]))
# PSNR_matrix_defined



In [44]:
out_images = glob.glob("/content/output_inbuilt/*.bmp")
out_images = sorted(out_images)
# print(out_images)
inp_images = glob.glob("/content/WavingTrees/b*.bmp")
inp_images = sorted(inp_images)
# print(inp_images)
PSNR_matrix_inbuilt = [] # # # A matrix that stores the PSNR values of each of the frames.
def max_psnr(original, output):
    original = cv2.imread(original)
    output = cv2.imread(output)
    value = PSNR(original, output)
    return value
for i in range(len(inp_images)):
  PSNR_matrix_inbuilt.append(max_psnr(inp_images[i], out_images[i]))
# PSNR_matrix_inbuilt

In [62]:
cum_error = 0
for i in range(len(PSNR_matrix_defined)):
  error = abs(PSNR_matrix_defined[i] - PSNR_matrix_inbuilt[i])/(PSNR_matrix_inbuilt[i])
  cum_error += error
print(f' Average Error in PSNR values for the frames is equal to {(cum_error/len(PSNR_matrix_defined))*100} %') # # # Difference between the PSNR values of user-defined and in-built functions.


 Average Error in PSNR values for the frames is equal to 0.026860152027581807 %


SSIM

In [52]:
# # Taken from ourcodeworld

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  # valid
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [65]:
# # # Calculates the Structural Similarity between the frames between the user-defined and in-built functions.
inp_images = glob.glob("/content/WavingTrees/b*.bmp")
inp_images = sorted(inp_images)
out_images_def = glob.glob("/content/output/*.bmp")
out_images_def = sorted(out_images_def)
# print(out_images)
out_images_inbuilt = glob.glob("/content/output_inbuilt/*.bmp")
out_images_inbuilt = sorted(out_images_inbuilt)
# print(inp_images)
SSIM_matrix_diff = []
SSIM_matrix_score = []

def max_ssim(original, output):
    original = cv2.imread(original)
    output = cv2.imread(output)
    diff = calculate_ssim(original, output)
    return diff

for i in range(len(out_images_def)):
  diff = max_ssim(out_images_def[i], out_images_inbuilt[i])
  SSIM_matrix_diff.append(diff)
  # SSIM_matrix_score.append(score)
print(f'Max Structural Similarity Index is equal to {max(SSIM_matrix_diff)}')

Max Structural Similarity Index is equal to 0.4683244064414526
